In [31]:
from __future__ import annotations
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage,SystemMessage
from dotenv import  load_dotenv
from pydantic import BaseModel,Field
from typing import List,TypedDict,Annotated,Literal
import operator
from langgraph.types import Send

In [32]:
load_dotenv()

True

In [33]:
# subclass defining of the pyydantic
class Task(BaseModel):
    id: int
    title: str
    goal : str = Field(..., description='One sentence describing what the reader should be able to do/understand after this section.')
    bullets: List[str] = Field(...,min_length=3,max_length=5,description='3–5 concrete, non-overlapping subpoints to cover in this section.')
    target_words : int = Field(...,description='Target word count for this section (120–450)')
    section_type: Literal["intro", "core", "examples", "checklist", "common_mistakes", "conclusion"] = Field(
        ...,
        description="Use 'common_mistakes' exactly once in the plan.",
    )

In [34]:
# Defining the pydantic class
class Plan(BaseModel):
    blog_title: str = Field(description="Title of the blog")
    audience : str = Field(description='Who this blog is for')
    tone : str = Field(description='writing tone (example : crisp,moody,practical ...)')
    tasks: List[Task] = Field(description='breaks the task into sections') 

In [35]:
class blog_writer(TypedDict):
    topic: str
    plan: Plan
    sections: Annotated[List[str], operator.add]
    final: str

In [36]:
# Defining the llm
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [37]:
def orchestrator(state: blog_writer) -> dict:
    planner = llm.with_structured_output(Plan)

    plan_result = planner.invoke([
        SystemMessage(content=(
                    "You are a senior technical writer and developer advocate. Your job is to produce a "
                    "highly actionable outline for a technical blog post.\n\n"
                    "Hard requirements:\n"
                    "- Create 5–7 sections (tasks) that fit a technical blog.\n"
                    "- Each section must include:\n"
                    "  1) goal (1 sentence: what the reader can do/understand after the section)\n"
                    "  2) 3–5 bullets that are concrete, specific, and non-overlapping\n"
                    "  3) target word count (120–450)\n"
                    "- Include EXACTLY ONE section with section_type='common_mistakes'.\n\n"
                    "Make it technical (not generic):\n"
                    "- Assume the reader is a developer; use correct terminology.\n"
                    "- Prefer design/engineering structure: problem → intuition → approach → implementation → "
                    "trade-offs → testing/observability → conclusion.\n"
                    "- Bullets must be actionable and testable (e.g., 'Show a minimal code snippet for X', "
                    "'Explain why Y fails under Z condition', 'Add a checklist for production readiness').\n"
                    "- Explicitly include at least ONE of the following somewhere in the plan (as bullets):\n"
                    "  * a minimal working example (MWE) or code sketch\n"
                    "  * edge cases / failure modes\n"
                    "  * performance/cost considerations\n"
                    "  * security/privacy considerations (if relevant)\n"
                    "  * debugging tips / observability (logs, metrics, traces)\n"
                    "- Avoid vague bullets like 'Explain X' or 'Discuss Y'. Every bullet should state what "
                    "to build/compare/measure/verify.\n\n"
                    "Ordering guidance:\n"
                    "- Start with a crisp intro and problem framing.\n"
                    "- Build core concepts before advanced details.\n"
                    "- Include one section for common mistakes and how to avoid them.\n"
                    "- End with a practical summary/checklist and next steps.\n\n"
                    "Output must strictly match the Plan schema."
                )
        ),
        HumanMessage(content=f"Topic: {state['topic']}")
    ])
    return {
        "plan": plan_result
    }

In [38]:
def fanout(state: blog_writer):
    return [
        Send(
            "worker",
            {
                "task": task,
                "plan": state["plan"],
                "topic": state["topic"]
            }
        )
        for task in state["plan"].tasks
    ]

In [39]:
def worker(payload: dict) -> dict:

    task = payload["task"]
    topic = payload["topic"]
    plan = payload["plan"]

    bullets_text = "\n- " + "\n- ".join(task.bullets)

    section_md = llm.invoke(
        [
            SystemMessage(
    content=(
        "You are a senior technical writer and developer advocate. Write ONE section of a technical blog post in Markdown.\n\n"
        "Hard constraints:\n"
        "- Follow the provided Goal and cover ALL Bullets in order (do not skip or merge bullets).\n"
        "- Stay close to the Target words (±15%).\n"
        "- Output ONLY the section content in Markdown (no blog title H1, no extra commentary).\n\n"
        "Technical quality bar:\n"
        "- Be precise and implementation-oriented (developers should be able to apply it).\n"
        "- Prefer concrete details over abstractions: APIs, data structures, protocols, and exact terms.\n"
        "- When relevant, include at least one of:\n"
        "  * a small code snippet (minimal, correct, and idiomatic)\n"
        "  * a tiny example input/output\n"
        "  * a checklist of steps\n"
        "  * a diagram described in text (e.g., 'Flow: A -> B -> C')\n"
        "- Explain trade-offs briefly (performance, cost, complexity, reliability).\n"
        "- Call out edge cases / failure modes and what to do about them.\n"
        "- If you mention a best practice, add the 'why' in one sentence.\n\n"
        "Markdown style:\n"
        "- Start with a '## <Section Title>' heading.\n"
        "- Use short paragraphs, bullet lists where helpful, and code fences for code.\n"
        "- Avoid fluff. Avoid marketing language.\n"
        "- If you include code, keep it focused on the bullet being addressed.\n"
    )
),
            HumanMessage(
                content=(
                    f"Blog: {plan.blog_title}\n"
                    f"Audience: {plan.audience}\n"
                    f"Tone: {plan.tone}\n"
                    f"Topic: {topic}\n\n"
                    f"Section: {task.title}\n"
                    f"Section type: {task.section_type}\n"
                    f"Goal: {task.goal}\n"
                    f"Target words: {task.target_words}\n"
                    f"Bullets:{bullets_text}\n"
                )
            ),
        ]
    ).content.strip()

    return {"sections": [section_md]}

In [40]:
from pathlib import Path
import re

def reducer(state: blog_writer) -> dict:
    title = state["plan"].blog_title
    body = "\n\n".join(state["sections"]).strip()
    final_md = f"# {title}\n\n{body}\n"

    safe_name = re.sub(r'[^a-zA-Z0-9_]', '_', title.lower())
    
    output_dir = Path("outputs")
    output_dir.mkdir(exist_ok=True)

    file_path = output_dir / f"{safe_name}.md"
    file_path.write_text(final_md, encoding="utf-8")

    print(f"Saved to: {file_path.resolve()}")

    return {"final": final_md}

In [41]:
graph = StateGraph(blog_writer)

graph.add_node('orchestrator', orchestrator)  # Fixed: 'orchestor' -> 'orchestrator'
graph.add_node('worker', worker)
graph.add_node('reducer', reducer)

graph.add_edge(START, 'orchestrator')  # Fixed: 'orchestor' -> 'orchestrator'
graph.add_conditional_edges('orchestrator', fanout, ['worker'])  # Fixed: 'orchestor' -> 'orchestrator'
graph.add_edge('worker', 'reducer')
graph.add_edge('reducer', END)

workflow = graph.compile()

In [42]:
out = workflow.invoke({"topic":'write a blog on self attention','sections':[]})

Saved to: C:\Users\akull\OneDrive\Desktop\Blog writing agent\outputs\demystifying_self_attention__the_core_of_modern_ai_models.md
